Inspect the data inside the truthtriplet files.

These are the upstream data files we will make the prong embed training data from.

To make the file that goes into this notebook run:

Script found in `ubdl/larflow/larflow/KeyPoints/test`
```
python run_prepalldata.py -tb -tri -adc wiremc --input-larlite [has mcinfo] --input-larcv [has wire/truth images] -o test.root
```

In [ ]:
import ROOT as rt
import numpy as np
from larcv import larcv
from larflow import larflow
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
import lardly
from lardly import DetectorOutline # load utility to draw TPC outline
detdata = DetectorOutline()
detlines = detdata.getlines(color=(10,10,10))


In [ ]:
rootfiles = ["test.root"]
rootfile_v = rt.std.vector("string")()
for rf in rootfiles:
    rootfile_v.push_back( rf )

In [ ]:
loader = larflow.spatialembed.PrepProngEmbed(rootfile_v)
loader._triplet_loader.load_tree()
loader.set_verbosity( larcv.msg.kDEBUG )

In [ ]:
ENTRY=3
nbytes = loader._triplet_loader.load_entry(ENTRY)
loader.make_subcluster_fragments()
print("number of bytes read: ",nbytes)

In [ ]:
# The triplet_v branch contains one instance of the larflow::prep::PrepMatchTriplet class
tripletdata = loader._triplet_loader.triplet_v.at(0)

In [ ]:
# we can get all the spacepoint data in nnumpy array form
data = tripletdata.make_triplet_ndarray(False)
print(data.keys())
for k in data.keys():
    print(k,": ",data[k].shape)

In [ ]:
# Plot instance ids

TRUE_ONLY=False
NU_ONLY=False
SHOW_ME_INDICES = []

if TRUE_ONLY:
    xyz = data['spacepoint_t'][ data['truetriplet_t']==1 ]
    trackid = data['instance_t'][ data['truetriplet_t']==1 ]
    origin  = data['origin_t'][ data['truetriplet_t']==1 ]
else:
    xyz     = data['spacepoint_t']
    trackid = data['instance_t']
    origin  = data['origin_t']
    
if xyz.shape[0]>200000:
    reduction_factor = 150e3/float(xyz.shape[0])
    keep = np.random.rand( xyz.shape[0] ) < reduction_factor
    xyz     = data['spacepoint_t'][keep==1]
    trackid = data['instance_t'][keep==1]
    origin  = data['origin_t'][keep==1]
    
print("xyz: ",xyz.shape)
print("trackids: ",trackid.shape)
print("origin: ",origin.shape)
print("origin ids: ",np.unique(origin))

if NU_ONLY:
    xyz = xyz[origin==1]
    trackid = trackid[origin==1]
    
print("xyz: ",xyz.shape)
print("trackids: ",trackid.shape)

uniqueids = np.unique(trackid)
print("number of unique ids: ",uniqueids.shape)

# PLOT LOOP
cluster_plots = []
for n,uid in enumerate(uniqueids):
    
    cluster_xyz = xyz[ trackid==uid ]
    #print("cluster for track[%d] has %d points"%(uid,cluster_xyz.shape[0]))
    randcol = np.random.randint(low=20,high=254,size=3)
    cluster_color = "rgb(%d,%d,%d)"%(randcol[0],randcol[1],randcol[2])
    c_plot = {
        "type":"scatter3d",
        "x": cluster_xyz[:,0],
        "y": cluster_xyz[:,1],
        "z": cluster_xyz[:,2],
        "mode":"markers",
        "name":"%d"%(uid),
        "marker":{"color":cluster_color,"size":2,"opacity":1.0},
    }
    
    if len(SHOW_ME_INDICES)>0:
        if uid not in SHOW_ME_INDICES:
            c_plot["marker"]["color"] = "rgb(25,25,25)"
            c_plot["marker"]["opacity"] = 0.2
            c_plot["marker"]["size"] = 1
        else:
            c_plot["marker"]["size"] = 2
    if uid==0:
        c_plot["marker"]["color"] = "rgb(170,170,170)"
        c_plot["marker"]["size"] = 1
        c_plot["marker"]["opacity"] = 0.1
    cluster_plots.append( c_plot )

detlines = detdata.getlines(color=(10,10,10))
plot_traces = detlines + cluster_plots

#PLOT VTX
if loader._triplet_loader.has_larbysmc:
    vtx = np.zeros((1,3))
    vtx[0,0] = loader._triplet_loader.vtx_sce_x
    vtx[0,1] = loader._triplet_loader.vtx_sce_y
    vtx[0,2] = loader._triplet_loader.vtx_sce_z
    print("VTX: ",vtx)
    plot_vtx = {
        "type":"scatter3d",
            "x": vtx[:,0],
            "y": vtx[:,1],
            "z": vtx[:,2],
            "mode":"markers",
            "name":"VTX"%(uid),
            "marker":{"color":"rgb(120,120,120)","size":10,"opacity":0.5},
        }
    plot_traces += [plot_vtx]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='All input larmatch hits',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()
